In [2]:
!pip install datasets
!pip install requests
!pip install python-dotenv


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import requests
import time
import math
import os
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from dotenv import load_dotenv

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()  # take environment variables from `.env`.

GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')

## Creating your own dataset

In [5]:
url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)
response.status_code

200

In [6]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/6460',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/6460/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/6460/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/6460/events',
  'html_url': 'https://github.com/huggingface/datasets/issues/6460',
  'id': 2017433899,
  'node_id': 'I_kwDODunzps54P5kr',
  'number': 6460,
  'title': "jsonlines files don't load with `load_dataset`",
  'user': {'login': 'serenalotreck',
   'id': 41377532,
   'node_id': 'MDQ6VXNlcjQxMzc3NTMy',
   'avatar_url': 'https://avatars.githubusercontent.com/u/41377532?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/serenalotreck',
   'html_url': 'https://github.com/serenalotreck',
   'followers_url': 'https://api.github.com/users/serenalotreck/followers',
   

In [7]:
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [8]:
def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("../datasets"),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [9]:
# Depending on your internet connection, this can take several minutes to run...
# fetch_issues()

In [10]:
df=pd.read_json("../datasets/datasets-issues.jsonl", lines=True)
df.head()

issues_dataset = Dataset.from_pandas(df)
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request'],
    num_rows: 6431
})

In [11]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/issues/3311
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/pull/3866
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/3866.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/3866', 'merged_at': '2022-03-08T17:37:01Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/3866.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/3866'}

>> URL: https://github.com/huggingface/datasets/issues/2148
>> Pull request: None



In [12]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map: 100%|██████████| 6431/6431 [00:02<00:00, 2197.96 examples/s]


In [13]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

In [14]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [ ]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

In [ ]:
issues_with_comments_dataset.push_to_hub("github-issues")

In [ ]:
remote_dataset = load_dataset("Shayawn/github-issues", split="train")
remote_dataset